In [1]:
from google.colab import drive
import zipfile
import os
import shutil

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to the ZIP file on Drive
zip_path = "/content/drive/MyDrive/deepfake.zip"

# Step 3: Create a temporary folder to extract
extract_path = "/content/temp"
os.makedirs(extract_path, exist_ok=True)

# Step 4: Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Step 5: Move extracted folder to /content/deepfake_clean
# Detect inner folder (in case it's named something else)
extracted_items = os.listdir(extract_path)
if len(extracted_items) == 1 and os.path.isdir(os.path.join(extract_path, extracted_items[0])):
    src_folder = os.path.join(extract_path, extracted_items[0])
else:
    src_folder = extract_path

dst_folder = "/content/deepfake_clean"
shutil.move(src_folder, dst_folder)

print(f"✅ Dataset extracted and moved to: {dst_folder}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset extracted and moved to: /content/deepfake_clean


In [2]:
!ls /content/deepfake_clean

Sample_fake_images  temp  test-20250112T065939Z-001  train-20250112T065955Z-001


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

# Define paths
train_dir = "/content/deepfake_clean/train-20250112T065955Z-001/train"
test_dir = "/content/deepfake_clean/test-20250112T065939Z-001/test"

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Load training and validation datasets
train_ds = image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary'  # since it's a binary classification
)

val_ds = image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary'
)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

Found 479 files belonging to 2 classes.
Found 499 files belonging to 2 classes.


In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Image augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

# Base model: EfficientNetB0
base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False  # Freeze base for now

# Complete model
model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)

# Summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.2),
])

# Set paths
dataset_path = "/content/deepfake_clean"
train_dir = os.path.join(dataset_path, "train-20250112T065955Z-001", "train")
test_dir = os.path.join(dataset_path, "test-20250112T065939Z-001", "test")

# Load datasets
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)
val_ds = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)

# Prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Build model
base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

model = tf.keras.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2)

# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop, lr_reduce]
)


Found 479 files belonging to 2 classes.
Using 384 files for training.
Found 479 files belonging to 2 classes.
Using 95 files for validation.
Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 70s 4s/step - accuracy: 0.4541 - loss: 0.7648 - val_accuracy: 0.5263 - val_loss: 0.7232 - learning_rate: 1.0000e-04
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/step - accuracy: 0.4674 - loss: 0.7728 - val_accuracy: 0.5053 - val_loss: 0.7078 - learning_rate: 1.0000e-04
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.4954 - loss: 0.7353 - val_accuracy: 0.5053 - val_loss: 0.6945 - learning_rate: 1.0000e-04
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.5064 - loss: 0.7474 - val_accuracy: 0.5368 - val_loss: 0.6839 - learning_rate: 1.0000e-04
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.5266 - loss: 0.7104 - val_accuracy: 0.5579 - val_loss: 0.6718 - learning_rate: 1.0000e-04
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.5400 - loss: 0.7072 - val

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load datasets
train_ds = image_dataset_from_directory(
    "/content/deepfake_clean/train-20250112T065955Z-001/train",
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
    validation_split=0.2,
    subset="training",
    seed=123,
)

val_ds = image_dataset_from_directory(
    "/content/deepfake_clean/train-20250112T065955Z-001/train",
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
    validation_split=0.2,
    subset="validation",
    seed=123,
)

test_ds = image_dataset_from_directory(
    "/content/deepfake_clean/test-20250112T065939Z-001/test",
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
)

# Prefetch to improve performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

# Load base model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = True

# Freeze bottom layers, unfreeze last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Build model
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)

# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stop, reduce_lr]
)

Found 479 files belonging to 2 classes.
Using 384 files for training.
Found 479 files belonging to 2 classes.
Using 95 files for validation.
Found 499 files belonging to 2 classes.
Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 80s 5s/step - accuracy: 0.6027 - loss: 0.6658 - val_accuracy: 0.4947 - val_loss: 0.7197 - learning_rate: 1.0000e-05
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.5730 - loss: 0.6861 - val_accuracy: 0.5053 - val_loss: 0.7136 - learning_rate: 1.0000e-05
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.5966 - loss: 0.6763 - val_accuracy: 0.5053 - val_loss: 0.7072 - learning_rate: 1.0000e-05
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.5587 - loss: 0.6627 - val_accuracy: 0.5053 - val_loss: 0.7011 - learning_rate: 1.0000e-05
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.6258 - loss: 0.6802 - val_accuracy: 0.5053 - val_loss: 0.6954 - learning_rate: 1.0000e-05
Epoch 6/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load datasets
train_ds = image_dataset_from_directory(
    "/content/deepfake_clean/train-20250112T065955Z-001/train",
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
    validation_split=0.2,
    subset="training",
    seed=123,
)

val_ds = image_dataset_from_directory(
    "/content/deepfake_clean/train-20250112T065955Z-001/train",
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
    validation_split=0.2,
    subset="validation",
    seed=123,
)

test_ds = image_dataset_from_directory(
    "/content/deepfake_clean/test-20250112T065939Z-001/test",
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
)

# Prefetch to improve performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

# Load base model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = True

# Freeze bottom layers, unfreeze last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Build model
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)

# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

model.save("/content/deepfake_efficientnet.keras", save_format="keras")

Found 479 files belonging to 2 classes.
Using 384 files for training.
Found 479 files belonging to 2 classes.
Using 95 files for validation.
Found 499 files belonging to 2 classes.
Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.5973 - loss: 0.6654 - val_accuracy: 0.6211 - val_loss: 0.6526 - learning_rate: 1.0000e-05
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 76s 4s/step - accuracy: 0.6217 - loss: 0.6541 - val_accuracy: 0.6526 - val_loss: 0.6484 - learning_rate: 1.0000e-05
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.6409 - loss: 0.6488 - val_accuracy: 0.6421 - val_loss: 0.6443 - learning_rate: 1.0000e-05
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.6576 - loss: 0.6528 - val_accuracy: 0.6421 - val_loss: 0.6402 - learning_rate: 1.0000e-05
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.6391 - loss: 0.6304 - val_accuracy: 0.6526 - val_loss: 0.6361 - learning_rate: 1.0000e-05
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 83s 5s/step

In [8]:
import os
os.path.exists("/content/deepfake_efficientnet.keras")

True

In [10]:
from google.colab import files

# Directly download the model file
files.download("/content/deepfake_efficientnet.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!pip install gradio

In [13]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the trained model
model = tf.keras.models.load_model("/content/deepfake_efficientnet.keras")

# Prediction function
def predict_image(img):
    img = img.resize((224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]
    label = "Fake" if prediction < 0.5 else "Real"
    confidence = 1 - prediction if prediction < 0.5 else prediction
    return f"{label} ({confidence:.2%} confidence)"

# Gradio interface with no examples
with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown(
        "<h1 style='text-align: center;'>🧠 Deepfake Image Detector</h1>"
        "<p style='text-align: center;'>Upload a face image and this model will detect whether it's Real or Fake.</p>"
    )

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Face Image")
            submit_btn = gr.Button("Detect Deepfake")
        with gr.Column():
            result_output = gr.Textbox(label="Prediction Result", placeholder="Result will be shown here")

    submit_btn.click(fn=predict_image, inputs=image_input, outputs=result_output)

# Launch the app
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a970980b5dd308648b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
